In [ ]:
from Algorithm import Gaussian_Process
from Algorithm import LA_Bayessian_Quadrature
from Algorithm import Mesh
from Algorithm import Kernels
import Target_Function as tf

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
opt_spy  = tf.Option('call', 0, 600, 0.1672, 0.15,  1, 100)   # SPDR S&P 500 ETF  (广义市场基准)
opt_aapl = tf.Option('call', 1, 200, 0.2613, 0.49,  1, 100)   # Apple
opt_tsla = tf.Option('call', 2, 350, 0.6398, 0.57,  1, 100)   # Tesla
opt_nvda = tf.Option('call', 3, 145, 0.3482, 0.24,  1, 100)   # NVIDIA
opt_amzn = tf.Option('call', 4, 210, 0.2902, 0.24,  1, 100)   # Amazon
opt_meta = tf.Option('call', 5, 700, 0.2924, 0.24,  1, 100)   # Meta
opt_msft = tf.Option('call', 6, 485, 0.1817, 0.15,  1, 100)   # Microsoft
opt_goog = tf.Option('call', 7, 165, 0.3226, 0.32,  1, 100)   # Google
opt_amd  = tf.Option('call', 8, 130, 0.4065, 0.32,  1, 100)   # AMD
opt_mstr = tf.Option('call', 9, 370, 0.5098, 0.49,  1, 100)   # MicroStrategy

def LABQ(D):
    options = [opt_spy, opt_aapl, opt_tsla, opt_nvda, opt_amzn,
               opt_meta, opt_msft, opt_goog, opt_amd, opt_mstr]
    portfolio = tf.OptionPortfolio(
        0.04,50,900,
        options[:D]
    )
    D = len(portfolio.options)
    func = portfolio.target_function
    xx = np.linspace(0., 1., 3)
    grids = np.meshgrid(*([xx] * D),)
    X = np.vstack([g.ravel() for g in grids]).T
    Y = np.array([func(x) for x in X])
    beta = np.concatenate([
    np.array([-0.5]),
    np.random.uniform(-2, 0, size=11 * D)
    ])
    mesh1 = Mesh.get_points_Sobol(X,_)
    GP = Gaussian_Process.GaussianProcess(Kernels.K_lin_mat2_10,beta,X,Y,mesh = mesh1)
    sobol = [8,7,6,5,4,3,3,2,2]
    n = [30,30,30,30,30,5,3,2,1]
    a, b, times, uncertainties = LA_Bayessian_Quadrature.ABC_D(
        func,
        GP,
        Mesh.get_points_Sobol,
        n = n[D-2],
        lambd = [30, 1],
        point_mesh = mesh1,
        sobol = sobol[D-2] #2^sobol points
    )
    """
    note of the result:
    Dimensions --> numbers of options
    Sobol --> 2^sobol points every step
    Steps --> how many cycle to run
    Time --> time of each step
    Result[mean, var] -->
    To standardize the values of each point for calculation purposes.
    The mean IS NOT final delta, it need to *Target_Function.portfolio.total

    Total_time(at the end of each dimension test) --> total time of each dimension test
    Uncertainty Rate --> sqrt(var) / mean * 100%

    There's Two test result below, the second one runs for 10hrs, but 7D can't even go 1Step, but 6D is worked out.
    Device: Macbook Pro with M3 Pro
    """
    return times,uncertainties

In [ ]:
def LABQ_alter(D):
    options = [opt_spy, opt_aapl, opt_tsla, opt_nvda, opt_amzn,
               opt_meta, opt_msft, opt_goog, opt_amd, opt_mstr]
    portfolio = tf.OptionPortfolio(
        0.04,50,900,
        options[:D]
    )
    D = len(portfolio.options)
    func = portfolio.target_function
    aaa = np.array([np.zeros(D)])
    X = Mesh.get_points_Sobol(aaa,0,D)
    Y = np.array([func(x) for x in X])
    beta = np.concatenate([
    np.array([-0.5]),
    np.random.uniform(-2, 0, size=11 * D)
    ])
    mesh1 = Mesh.get_points_Sobol(X,_)
    GP = Gaussian_Process.GaussianProcess(Kernels.K_lin_mat2_10,beta,X,Y,mesh = mesh1)
    sobol = [8,7,6,5,4,3,3]
    n = [30,30,30,30,30,20,10]
    a, b, times, uncertainties = LA_Bayessian_Quadrature.ABC_D(
        func,
        GP,
        Mesh.get_points_Sobol,
        n = n[D-2],
        lambd = [30, 1],
        point_mesh = mesh1,
        sobol = sobol[D-2] #2^sobol points
    )
    return times,uncertainties

In [ ]:
t = []
un = []

In [ ]:
ti, unce = LABQ_alter(2)
t.append(ti)
un.append(unce)

In [ ]:
ti, unce = LABQ_alter(3)
t.append(ti)
un.append(unce)

In [ ]:
ti, unce = LABQ_alter(4)
t.append(ti)
un.append(unce)

In [ ]:
ti, unce = LABQ_alter(5)
t.append(ti)
un.append(unce)

In [ ]:
ti, unce = LABQ_alter(6)
t.append(ti)
un.append(unce)

In [ ]:
ti, unce = LABQ_alter(7)
t.append(ti)
un.append(unce)

In [ ]:
ti, unce = LABQ_alter(8)
t.append(ti)
un.append(unce)

In [ ]:
dimension = 8
avg_time = []
for i in range(dimension - 1):
    plt.plot(range(len(un[i])), un[i], label=f"{i+2}D")
    avg_time.append(np.mean(t[i]))
plt.title("Uncertainty Rate")
plt.legend()
plt.show()
plt.plot(range(2, dimension + 1),avg_time)
plt.title("Average Time")
plt.show()
